## Sample game

In [1]:
import sys
import time
from copy import deepcopy
from IPython.display import clear_output

if '..' not in sys.path: sys.path.append('..')
from src.utils.common import *
from src.dnd.actions import *
from src.dnd.units import *
from src.dnd.game_utils import *
from src.dnd.game_board import DnDBoard
from src.agent.agent import DnDAgent
from src.agent.agent_utils import get_states, get_default_action_resolver

In [2]:
def create_game():
    game = DnDBoard((5, 5))
    ally = GenericSoldier("Ally soldier", attack_damage=25)
    ally2 = GenericSoldier("Ally ranger", attack_damage=25, health=50, range=4)
    enemy = GenericSoldier("Enemy soldier", attack_damage=25)
    enemy2 = GenericSoldier("Enemy ranger", attack_damage=25, health=50, range=4)

    place_unit_randomly(game, ally, 0)
    place_unit_randomly(game, enemy, 1)
    place_unit_randomly(game, ally2, 0)
    place_unit_randomly(game, enemy2, 1)

    game.initialize_game()
    action_resolver = get_default_action_resolver(game)

    color_map = {
        ally: "Green",
        enemy: "Red",
        ally2: "Blue",
        enemy2: "Purple"
    }

    return game, color_map, action_resolver

In [3]:
def create_pair_game(ref: DnDBoard):
    game = DnDBoard((5, 5))
    ally = GenericSoldier("Ally soldier", attack_damage=25)
    ally2 = GenericSoldier("Ally ranger", attack_damage=25, health=50, range=4)
    enemy = GenericSoldier("Enemy soldier", attack_damage=25)
    enemy2 = GenericSoldier("Enemy ranger", attack_damage=25, health=50, range=4)

    names_to_units = { unit.name: unit for unit in ref.units }
    game.place_unit(ally, to_tuple(ref.get_unit_position(names_to_units[ally.name])), 1)
    game.place_unit(ally2, to_tuple(ref.get_unit_position(names_to_units[ally2.name])), 1)
    game.place_unit(enemy, to_tuple(ref.get_unit_position(names_to_units[enemy.name])), 0)
    game.place_unit(enemy2, to_tuple(ref.get_unit_position(names_to_units[enemy2.name])), 0)

    game.initialize_game()
    game.turn_order = ref.turn_order.copy()
    action_resolver = get_default_action_resolver(game)

    color_map = {
        ally: "Green",
        enemy: "Red",
        ally2: "Blue",
        enemy2: "Purple"
    }

    return game, color_map, action_resolver

In [4]:
fms_7_ch = ['Ally units', 'Enemy units', 'Current unit', 'Movement speed', 'Attack range', 'Attack damage', 'Health']

In [5]:
agent_path_1 = '../config-log/2v2-0/trained-agents/agent-gen26-10.3i'
agent_path_2 = '../config-log/2v2-0/trained-agents/agent-gen15-10.2i'
agent1 = DnDAgent.load_agent(agent_path_1)
agent1.epsilon = 0
indices1 = get_observation_indices(fms_7_ch if agent1.in_channels == len(fms_7_ch) else None)
agent2 = DnDAgent.load_agent(agent_path_2)
agent2.epsilon = 0
indices2 = get_observation_indices(fms_7_ch if agent2.in_channels == len(fms_7_ch) else None)

In [6]:
def play_loop(agent1, agent2, game: DnDBoard, color_map, random_action_resolver=None, manual_input=False, delay=0.5):
    game_over = False
    iter_count = 0

    print_game(game, color_map)
    while not game_over:
        try:
            unit, player_id = game.get_current_unit()

            iter_count += 1

            # sleeping 
            if manual_input:
                command = input()
                if command == 'stop':
                    raise KeyboardInterrupt()
                elif command == 'continue':
                    manual_input = False
            else:
                time.sleep(delay)
            
            clear_output(wait=True)
            print(f'Iteration: {iter_count}')

            agent = agent1 if player_id == 0 else agent2
            _, _, new_coords, action = get_states(game, agent, random_action_resolver)
            _, game_over = take_turn(game, new_coords, action, color_map, True)
        except KeyboardInterrupt:
            print(f'\nGame interrupted after {iter_count} iterations')
            return None

    winner = 0 if len(game.players_to_units[1]) == 0 else 1
    print(f'\nGame over in {iter_count} iterations. Winner: agent #{winner + 1}')

    return iter_count, winner

Start new game:

In [7]:
counter = []

In [8]:
# game, color_map, action_resolver = create_game()
# 
# ic, win = play_loop(agent1, agent1, game, color_map, action_resolver, False, 1)
# 
# counter.append(win)

In [9]:
def play_loop_fast(agent1, agent2, game: DnDBoard, random_action_resolver=None, iter_limit=1000, indices1=None, indices2=None):
    game_over = False
    iter_count = 0

    while not game_over:
        if iter_count > iter_limit: return iter_count, -1
        
        unit, player_id = game.get_current_unit()
        iter_count += 1

        agent, indices = (agent1, indices1)  if player_id == 0 else (agent2, indices2)
        _, _, new_coords, action = get_states(game, agent, random_action_resolver, state_indices=indices)
        _, game_over = game.take_turn(new_coords, action, True)

    winner = 0 if len(game.players_to_units[1]) == 0 else 1

    return iter_count, winner

In [10]:
from tqdm.auto import tqdm

for i in tqdm(range(50000)):
    game, color_map, action_resolver = create_game()
    game2, color_map2, action_resolver2 = create_pair_game(game)
    
    ic, win = play_loop_fast(agent1, agent2, game, action_resolver, indices1=indices1, indices2=indices2)
    ic2, win2 = play_loop_fast(agent1, agent2, game2, action_resolver2, indices1=indices1, indices2=indices2)
    
    counter.append(win)
    counter.append(win2)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [11]:
def path_to_agent_name(path):
    split_path = path.split('/')
    return split_path[-3] + '/' + split_path[-1]

npcounter = np.array(counter)
wins_one = npcounter[::2]
wins_pair = npcounter[1::2]

print(
f'''|           | `{path_to_agent_name(agent_path_1)}`  | `{path_to_agent_name(agent_path_2)}` |
| --------- | -------  | ------- |
| wins      | {np.sum(npcounter == 0)}    | {np.sum(npcounter == 1)}   |
| pair-wins | {np.sum(np.logical_and(wins_one == 0, wins_pair == 0))}     | {np.sum(np.logical_and(wins_one == 1, wins_pair == 1))}   |

Games timed out: {np.sum(npcounter == -1)}
''')

|           | `2v2-0/agent-gen26-10.3i`  | `2v2-0/agent-gen15-10.2i` |
| --------- | -------  | ------- |
| wins      | 54739    | 45234   |
| pair-wins | 13255     | 8502   |

Games timed out: 27

